# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('custChurn').getOrCreate()

In [3]:
from pyspark.ml.classification import LogisticRegression

In [4]:
df = spark.read.csv('./data/customer_churn.csv', header=True, inferSchema=True)

In [7]:
df.head(1)

[Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1)]

In [8]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [9]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [12]:
all_data = df.select(
     'Age',
     'Total_Purchase',
     'Years',
     'Num_Sites',
     'Location',
     'Company'
)

In [13]:
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder, VectorIndexer

In [14]:
location_indexer = StringIndexer(inputCol='Location', outputCol='LocationIdx')
location_encoder = OneHotEncoder(inputCol='LocationIdx', outputCol='LocationVec')

In [15]:
company_indexer = StringIndexer(inputCol='Company', outputCol='CompanyIdx')
company_encoder = OneHotEncoder(inputCol='CompanyIdx', outputCol='CompanyVec')

In [39]:
assembler = VectorAssembler(inputCols=[
     'Age',
     'Total_Purchase',
     'Years',
     'Num_Sites',
    ],
    outputCol='features')

In [40]:
from pyspark.ml import Pipeline

In [41]:
logmodel = LogisticRegression(featuresCol='features', labelCol='Churn')

In [42]:
pipe = Pipeline(stages=[
    assembler,
    logmodel
])

In [43]:
train_set, test_set = df.randomSplit([0.7,0.3])

In [44]:
trained_model = pipe.fit(train_set)

In [45]:
results = trained_model.transform(test_set)

In [46]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [47]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn')

In [49]:
AUC = evaluator.evaluate(results)

In [54]:
AUC

0.736890756302521

In [38]:
df_test = spark.read.csv('./data/new_customers.csv', header=True, inferSchema=True)

In [57]:
final_results = trained_model.transform(df_test)

In [61]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



In [ ]:
# Same results even if we dont train on the entire thing. Jose did that. 